# Enhanced Geometry and Mesh for Tank plus Nozzle 

This notebook demonstrates the creation of 2D and 3D meshes for a tank with nozzle geometry using GMSH. The notebook covers:

## Objectives:
1. ✅ Generate quadrilateral mesh more adapted to the flow
2. ✅ **NEW**: Revolve mesh around axis to create a 3D wedge mesh
3. ✅ **NEW**: Enhanced documentation

## Key Features:
- **2D Mesh Generation**: Multiple approaches including basic geometry, transfinite meshing
- **3D Mesh Generation**: Revolution of 2D mesh to create 3D wedge geometries
- **Flow-Adapted Meshes**: Structured quadrilateral meshes suitable for CFD applications
- **Physical Groups**: Proper boundary condition assignment for inlet, outlet, walls, and axis

## Applications:
- Metal hydride storage tank analysis
- Computational fluid dynamics (CFD) simulations
- Heat and mass transfer studies
- Tank filling/emptying processes

## Import Packages

The following cell imports the GMSH library with fallback options to ensure compatibility across different Julia environments.

In [ ]:
# Enhanced GMSH import with multiple fallback options
println("Attempting to import GMSH...")

try
    using Gmsh: gmsh
    println("GMSH imported successfully via Gmsh.jl!")
catch
    try
        using gmsh
        println("GMSH imported successfully via gmsh!")
    catch e
        println("Error importing GMSH: ", e)
        println("Please install GMSH using: Pkg.add(\"Gmsh\")")
        rethrow(e)
    end
end

## Section 1: Introduction and Geometry Overview

This notebook demonstrates various approaches to mesh generation for a tank with nozzle geometry, which is commonly used in metal hydride storage systems. The geometry consists of:

- **Main Tank**: Rectangular section representing the main storage volume
- **Nozzle**: Smaller rectangular section for gas inlet/outlet

### Key Considerations:
- **Mesh Quality**: Quadrilateral elements preferred for CFD applications
- **Boundary Layers**: Fine mesh near walls for accurate flow prediction
- **Structured Mesh**: Transfinite meshing for better element quality
- **3D Extension**: Revolution capability for axisymmetric geometries

The following sections will demonstrate different meshing approaches, from basic to advanced techniques.

## Section 2: Complete Geometry (Upper and Lower Half)

This section creates a complete tank with nozzle geometry including both upper and lower halves. This approach is useful for:
- Full 2D simulations
- Symmetric flow analysis
- Complete geometry visualization

### Parameters:
- `L = 1.0`: Tank length
- `R = 0.1`: Tank half-height
- `Lin = -0.1*L`: Nozzle extension length
- `Rin = 0.2*R`: Nozzle half-height
- `lc1 = 0.001`: Fine mesh density (near walls)
- `lc2 = 0.05`: Coarse mesh density (far field)

In [ ]:
#..1/7: initialize gmsh 
gmsh.initialize()

#..2/7: generate geometry 
gmsh.option.setNumber("General.Terminal", 2)
gmsh.model.add("tankNozzle_complete")

#..geometry parameters 
L = 1.0 
R = 0.1
Lin = -0.1*L 
Rin = 0.2*R 

#..set mesh density parameter 
lc1 = 0.001 # fine mesh near walls
lc2 = 0.05  # coarse mesh in far field

#..define points via (x,y,z) coordinates 
p1 = gmsh.model.geo.addPoint(0, -R, 0, lc1, 1)
p2 = gmsh.model.geo.addPoint(L, -R,  0, lc2, 2)
p3 = gmsh.model.geo.addPoint(L, R, 0, lc2, 3)
p4 = gmsh.model.geo.addPoint(0, R, 0, lc1, 4)
p5 = gmsh.model.geo.addPoint(0, Rin,  0, lc1, 5)
p6 = gmsh.model.geo.addPoint(Lin, Rin, 0, lc1, 6)
p7 = gmsh.model.geo.addPoint(Lin, -Rin, 0, lc1, 7)
p8 = gmsh.model.geo.addPoint(0, -Rin, 0, lc1, 8)

#..define edges by connecting point labels pairwise  
l1 = gmsh.model.geo.addLine(1, 2, 1)
l2 = gmsh.model.geo.addLine(2, 3, 2)
l3 = gmsh.model.geo.addLine(3, 4, 3)
l4 = gmsh.model.geo.addLine(4, 5, 4)
l5 = gmsh.model.geo.addLine(5, 6, 5)
l6 = gmsh.model.geo.addLine(6, 7, 6)
l7 = gmsh.model.geo.addLine(7, 8, 7)
l8 = gmsh.model.geo.addLine(8, 1, 8)

#..define closed loop by connecting edge labels  
loop = gmsh.model.geo.addCurveLoop([1, 2, 3, 4, 5, 6, 7, 8], 1)

#..define surface by closed loop 
surf = gmsh.model.geo.addPlaneSurface([1], 1)

#..3/7: synchronize the CAD model 
gmsh.model.geo.synchronize()

#..4/7: assign physical groups for boundary conditions
gmsh.model.addPhysicalGroup(1, [l1, l2, l3, l4, l5, l7, l8], -1, "wall")
gmsh.model.addPhysicalGroup(1, [l6], -1, "inlet")
gmsh.model.addPhysicalGroup(2, [surf], -1, "omega")

#..5/7: generate two-dimensional mesh 
# Enable recombination to create quadrilateral elements
gmsh.model.mesh.setRecombine(2,1)
gmsh.model.mesh.generate(2)

#..6/7: write mesh to file and visualize 
if (true) gmsh.write("tankAndNozzle_complete.msh") end 
if (true) gmsh.fltk.run() end 

#..7/7: finalize gmsh 
gmsh.finalize()

## Section 3: Upper Half Geometry (Axisymmetric)

This section creates only the upper half of the tank geometry, which is suitable for:
- Axisymmetric simulations
- Reduced computational domain
- Foundation for 3D revolution

### Key Features:
- **Axis boundary**: Bottom edge serves as axis of symmetry
- **Outlet boundary**: Right edge for gas outlet
- **Reduced domain**: Half the computational volume
- **Physical groups**: Proper boundary condition assignment

In [ ]:
#..1/7: initialize gmsh 
gmsh.initialize()

#..2/7: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("tankNozzle_upper")

#..geometry parameters 
L = 1.0 
R = 0.1
Lin = -0.1*L 
Rin = 0.3*R 

#..set mesh density parameter 
lc1 = 0.005  # fine mesh near walls
lc2 = 0.05   # coarse mesh in far field

#..define points via (x,y,z) coordinates (upper half only)
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc1, 1)      # axis point
p2 = gmsh.model.geo.addPoint(L, 0,  0, lc2, 2)     # axis point
p3 = gmsh.model.geo.addPoint(L, R, 0, lc2, 3)      # tank corner
p4 = gmsh.model.geo.addPoint(0, R, 0, lc1, 4)      # tank corner
p5 = gmsh.model.geo.addPoint(0, Rin,  0, lc1, 5)   # nozzle connection
p6 = gmsh.model.geo.addPoint(Lin, Rin, 0, lc1, 6)  # nozzle corner
p7 = gmsh.model.geo.addPoint(Lin, 0, 0, lc1, 7)    # nozzle axis point

#..define edges by connecting point labels pairwise  
l1 = gmsh.model.geo.addLine(1, 2, 1)  # axis (tank)
l2 = gmsh.model.geo.addLine(2, 3, 2)  # outlet
l3 = gmsh.model.geo.addLine(3, 4, 3)  # wall
l4 = gmsh.model.geo.addLine(4, 5, 4)  # wall
l5 = gmsh.model.geo.addLine(5, 6, 5)  # wall
l6 = gmsh.model.geo.addLine(6, 7, 6)  # inlet
l7 = gmsh.model.geo.addLine(7, 1, 7)  # axis (nozzle)

#..define closed loop by connecting edge labels  
loop = gmsh.model.geo.addCurveLoop([1, 2, 3, 4, 5, 6, 7], 1)

#..define surface by closed loop 
surf = gmsh.model.geo.addPlaneSurface([1], 1)

#..3/7: synchronize the CAD model 
gmsh.model.geo.synchronize()

#..4/7: assign physical groups for boundary conditions
gmsh.model.addPhysicalGroup(1, [l1, l7], -1, "axis")
gmsh.model.addPhysicalGroup(1, [l2], -1, "outlet")
gmsh.model.addPhysicalGroup(1, [l3,l4,l5], -1, "wall")
gmsh.model.addPhysicalGroup(1, [l6], -1, "inlet")
gmsh.model.addPhysicalGroup(2, [surf], -1, "omega")

#..5/7: generate two-dimensional mesh 
# Enable recombination to create quadrilateral elements
gmsh.model.mesh.setRecombine(2,1) 
gmsh.model.mesh.generate(2)

#..6/7: write mesh to file and visualize 
if (true) gmsh.write("tankAndNozzle_upper.msh") end 
if (true) gmsh.fltk.run() end 

#..7/7: finalize gmsh 
gmsh.finalize()

## Section 5: Using OpenCascade (Alternative Approach)

This section demonstrates an alternative geometry creation method using GMSH's OpenCascade (OCC) kernel instead of the built-in geometry kernel.

### Advantages of OpenCascade:
- **Boolean Operations**: Easy union, intersection, and difference operations
- **Primitive Shapes**: Built-in rectangles, circles, and other shapes
- **Robust Geometry**: Better handling of complex geometries
- **CAD Integration**: Compatible with CAD file formats

### Method:
- Create tank and nozzle as separate rectangles
- Use boolean operations to combine them
- Apply mesh generation algorithms

In [ ]:
#..geometry parameters 
L = 1.0 
R = 0.1
Lin = 0.1*L 
Rin = 0.3*R 

# Initialize GMSH
gmsh.initialize()
gmsh.option.setNumber("General.Verbosity", 2)
gmsh.model.add("tankNozzle_OCC")

# Create geometry using OpenCascade
dim = 2
# Create tank as rectangle
tank_tag = gmsh.model.occ.addRectangle(0, -R/2, 0, L, R)
# Create nozzle as rectangle
nozzle_tag = gmsh.model.occ.addRectangle(-Lin, -Rin/2, 0, Lin, Rin)

# Synchronize OCC model
gmsh.model.occ.synchronize()

# Assign physical groups
# Get all curves after synchronization
all_curves = gmsh.model.getEntities(1)
curve_tags = [c[2] for c in all_curves]

# Assign physical groups (simplified approach)
gmsh.model.addPhysicalGroup(1, curve_tags[1:end-1], -1, "wall")
gmsh.model.addPhysicalGroup(1, [curve_tags[end]], -1, "inlet")

# Get all surfaces
all_surfaces = gmsh.model.getEntities(2)
surface_tags = [s[2] for s in all_surfaces]
gmsh.model.addPhysicalGroup(2, surface_tags, -1, "omega")

# Set mesh options
gmsh.option.setNumber("Mesh.Algorithm", 11)  # quadrilateral mesh
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 20)
gmsh.option.setNumber("Mesh.MeshSizeMax", 0.01)

# Generate mesh
gmsh.model.mesh.generate(dim)

# Write and visualize
if (true) gmsh.write("tankAndNozzle_OCC.msh") end 
if (true) gmsh.fltk.run() end 

gmsh.finalize()

## Section 6: Transfinite Meshing for High-Quality Structured Meshes

Transfinite meshing creates structured meshes with precise control over element distribution and quality. This is particularly important for CFD applications where mesh quality directly affects solution accuracy.

### Key Features:
- **Node Distribution Control**: Specify exact number of nodes on each edge
- **Geometric Progression**: Control mesh refinement near boundaries
- **Structured Quadrilaterals**: High-quality elements aligned with flow
- **Multi-Block Approach**: Divide geometry into structured blocks

### Applications:
- **Boundary Layer Meshing**: Fine mesh near walls
- **Flow-Aligned Meshes**: Elements aligned with expected flow direction
- **High-Quality CFD**: Reduced numerical diffusion and better convergence

### Implementation Strategy:
1. Divide complex geometry into simple quadrilateral blocks
2. Apply transfinite constraints to each block
3. Use geometric progression for boundary layer refinement
4. Combine blocks to form complete mesh

In [ ]:
#..1/7: initialize gmsh 
gmsh.initialize()

#..2/7: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("tankNozzle_transfinite")

#..geometry parameters 
L = 1.0 
R = 0.1
Lin = -0.1*L 
Rin = 0.3*R 

#..set mesh density parameter 
lc1 = 0.005
lc2 = 0.05 

#..define points for multi-block structured mesh
p1  = gmsh.model.geo.addPoint(0,   -R,   0, lc1, 1)
p2  = gmsh.model.geo.addPoint(L,   -R,   0, lc2, 2)
p3  = gmsh.model.geo.addPoint(L,   -Rin, 0, lc2, 3)
p4  = gmsh.model.geo.addPoint(L,    Rin, 0, lc2, 4)
p5  = gmsh.model.geo.addPoint(L,    R,   0, lc2, 5)
p6  = gmsh.model.geo.addPoint(0,    R,   0, lc1, 6)
p7  = gmsh.model.geo.addPoint(0,    Rin, 0, lc1, 7)
p8  = gmsh.model.geo.addPoint(Lin,  Rin, 0, lc1, 8)
p9  = gmsh.model.geo.addPoint(Lin, -Rin, 0, lc1, 9)
p10 = gmsh.model.geo.addPoint(0,   -Rin, 0, lc1, 10)

#..define edges by connecting point labels pairwise  
l1 = gmsh.model.geo.addLine(1, 2, 1)
l2 = gmsh.model.geo.addLine(2, 3, 2)
l3 = gmsh.model.geo.addLine(3, 4, 3)
l4 = gmsh.model.geo.addLine(4, 5, 4)
l5 = gmsh.model.geo.addLine(5, 6, 5)
l6 = gmsh.model.geo.addLine(6, 7, 6)
l7 = gmsh.model.geo.addLine(7, 8, 7)
l8 = gmsh.model.geo.addLine(8, 9, 8)
l9 = gmsh.model.geo.addLine(9, 10, 9)
l10 = gmsh.model.geo.addLine(10, 1, 10)
l11 = gmsh.model.geo.addLine(10, 7, 11)
l12 = gmsh.model.geo.addLine(3, 10, 12)
l13 = gmsh.model.geo.addLine(7,  4, 13)

#..define curved loops by connecting edge labels  
loop1 = gmsh.model.geo.addCurveLoop([1, 2, 12, 10], 1)     # bottom block
loop2 = gmsh.model.geo.addCurveLoop([-12, 3, -13, -11], 2) # middle block
loop3 = gmsh.model.geo.addCurveLoop([13, 4, 5, 6], 3)      # top block
loop4 = gmsh.model.geo.addCurveLoop([9,11,7,8], 4)         # nozzle block

#..define surfaces by curved loops 
surf1 = gmsh.model.geo.addPlaneSurface([1], 1)
surf2 = gmsh.model.geo.addPlaneSurface([2], 2)
surf3 = gmsh.model.geo.addPlaneSurface([3], 3)
surf4 = gmsh.model.geo.addPlaneSurface([4], 4)

#..APPLY TRANSFINITE CONSTRAINTS
# Block 1 (bottom): 
gmsh.model.geo.mesh.setTransfiniteCurve(1, 10)   # axial direction
gmsh.model.geo.mesh.setTransfiniteCurve(12, 10)  # axial direction
gmsh.model.geo.mesh.setTransfiniteCurve(2, 5)    # radial direction
gmsh.model.geo.mesh.setTransfiniteCurve(10, 5)   # radial direction

# Block 2 (middle):
gmsh.model.geo.mesh.setTransfiniteCurve(12, 10)  # axial direction
gmsh.model.geo.mesh.setTransfiniteCurve(13, 10)  # axial direction
gmsh.model.geo.mesh.setTransfiniteCurve(3, 5)    # radial direction
gmsh.model.geo.mesh.setTransfiniteCurve(11, 5)   # radial direction

# Block 3 (top):
gmsh.model.geo.mesh.setTransfiniteCurve(13, 10)  # axial direction 
gmsh.model.geo.mesh.setTransfiniteCurve(5, 10)   # axial direction
gmsh.model.geo.mesh.setTransfiniteCurve(4, 5)    # radial direction
gmsh.model.geo.mesh.setTransfiniteCurve(6, 5)    # radial direction

# Block 4 (nozzle):
gmsh.model.geo.mesh.setTransfiniteCurve(9, 10)   # axial direction (nozzle)
gmsh.model.geo.mesh.setTransfiniteCurve(7, 10)   # axial direction (nozzle)
gmsh.model.geo.mesh.setTransfiniteCurve(11, 5)   # radial direction
gmsh.model.geo.mesh.setTransfiniteCurve(8, 5)    # inlet

#..apply transfinite surfaces (structured mesh)
gmsh.model.geo.mesh.setTransfiniteSurface(1) 
gmsh.model.geo.mesh.setTransfiniteSurface(2)
gmsh.model.geo.mesh.setTransfiniteSurface(3) 
gmsh.model.geo.mesh.setTransfiniteSurface(4)

#..enable recombination for quadrilateral elements
gmsh.model.geo.mesh.setRecombine(2,1) 
gmsh.model.geo.mesh.setRecombine(2,2)
gmsh.model.geo.mesh.setRecombine(2,3) 
gmsh.model.geo.mesh.setRecombine(2,4)

#..3/7: synchronize the CAD model 
gmsh.model.geo.synchronize()

#..4/7: assign physical groups
gmsh.model.addPhysicalGroup(1, [l1,l2,l3,l4,l5,l6,l7,l9,l10], -1, "wall")
gmsh.model.addPhysicalGroup(1, [l8], -1, "inlet")
gmsh.model.addPhysicalGroup(2, [surf1,surf2,surf3,surf4], -1, "omega")

#..5/7: generate two-dimensional mesh
gmsh.model.mesh.generate(2)

#..6/7: write mesh to file and visualize
if (true) gmsh.write("tankAndNozzle_transfinite.msh") end 
if (true) gmsh.fltk.run() end 

println("Transfinite structured mesh generated successfully!")
println("Mesh blocks: 4 (bottom, middle, top, nozzle)")
println("Element type: Quadrilateral (structured)")

#..7/7: finalize gmsh 
gmsh.finalize()

## Section 6: Utility Functions and Mesh Analysis

This section provides utility functions for mesh analysis and quality assessment.

In [ ]:
"""
Utility function to analyze mesh quality and statistics
"""
function analyze_mesh_quality(filename::String)
    gmsh.initialize()
    gmsh.open(filename)
    
    # Get mesh statistics
    nodes = gmsh.model.mesh.getNodes()
    elements_2d = gmsh.model.mesh.getElements(2)
    elements_3d = gmsh.model.mesh.getElements(3)
    
    println("=== Mesh Statistics ===")
    println("Nodes: $(length(nodes[1]))")
    
    if length(elements_2d[1]) > 0
        println("2D Elements: $(length(elements_2d[2][1]))")
        println("2D Element types: $(elements_2d[1])")
    end
    
    if length(elements_3d[1]) > 0
        println("3D Elements: $(length(elements_3d[2][1]))")
        println("3D Element types: $(elements_3d[1])")
    end
    
    # Get physical groups
    physical_groups = gmsh.model.getPhysicalGroups()
    println("\nPhysical Groups:")
    for (dim, tag) in physical_groups
        name = gmsh.model.getPhysicalName(dim, tag)
        println("  $(name): dimension $(dim), tag $(tag)")
    end
    
    gmsh.finalize()
end

"""
Function to create a simple mesh quality report
"""
function create_mesh_comparison()
    meshes = [
        "tankAndNozzle_complete.msh",
        "tankAndNozzle_upper.msh", 
        "tankAndNozzle_3D_wedge.msh",
        "tankAndNozzle_transfinite.msh",
        "tankAndNozzle_3D_advanced.msh"
    ]
    
    println("=== Mesh Comparison Report ===")
    for mesh in meshes
        if isfile(mesh)
            println("\n--- $(mesh) ---")
            try
                analyze_mesh_quality(mesh)
            catch e
                println("Error analyzing $(mesh): $(e)")
            end
        else
            println("\n--- $(mesh) ---")
            println("File not found (not generated yet)")
        end
    end
end

# Run the comparison
create_mesh_comparison()

## Section 5: 3D Wedge Mesh from Transfinite Base - **NEW FEATURE (WIP)**

This section implements the second todo item: **revolve mesh around axis to create a 3D wedge mesh**. 

### Why Transfinite Base is Essential for 3D Revolution:
Transfinite meshing provides the structured foundation needed for proper 3D revolution. Here's why:

- **Structured 2D Base**: Transfinite creates perfectly aligned quadrilaterals
- **Revolution Compatibility**: Structured meshes revolve cleanly into hexahedral elements
- **Quality Preservation**: Maintains element quality through revolution
- **Layer Control**: Enables precise control of circumferential layers

### Key Features:
- **Revolution**: 2D transfinite mesh rotated around the axis
- **Wedge Angle**: Small angle (e.g., 5° or 10°) for efficient 3D simulation
- **Structured 3D Mesh**: Maintains quadrilateral structure in 3D (hexahedral elements)
- **Periodic Boundaries**: Front and back faces for periodic boundary conditions

### Applications:
- 3D CFD simulations with reduced computational cost
- Axisymmetric flows with small 3D perturbations
- Validation of 2D axisymmetric results
- High-quality 3D mesh for metal hydride storage analysis

### Method:
1. Create transfinite 2D structured mesh (upper half geometry)
2. Use `gmsh.model.geo.revolve()` to create 3D wedge
3. Apply appropriate 3D boundary conditions
4. Generate structured 3D mesh

In [ ]:
#..1/9: initialize gmsh 
gmsh.initialize()

#..2/9: generate high-quality 2D transfinite mesh (upper half)
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("tankNozzle_3D_wedge")

#..geometry parameters 
L = 1.0 
R = 0.1
Lin = -0.1*L 
Rin = 0.3*R 

#..mesh density parameters
lc1 = 0.01   # fine mesh near walls
lc2 = 0.05   # coarse mesh in far field

#..create UPPER HALF geometry for revolution (starts from axis y=0)
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc1, 1)      # axis origin
p2 = gmsh.model.geo.addPoint(L, 0, 0, lc2, 2)      # axis end (tank)
p3 = gmsh.model.geo.addPoint(L, R, 0, lc2, 3)      # tank corner
p4 = gmsh.model.geo.addPoint(0, R, 0, lc1, 4)      # tank top
p5 = gmsh.model.geo.addPoint(0, Rin, 0, lc1, 5)    # nozzle connection
p6 = gmsh.model.geo.addPoint(Lin, Rin, 0, lc1, 6)  # nozzle corner
p7 = gmsh.model.geo.addPoint(Lin, 0, 0, lc1, 7)    # nozzle axis

#..create lines
l1 = gmsh.model.geo.addLine(1, 2, 1)  # tank axis
l2 = gmsh.model.geo.addLine(2, 3, 2)  # tank outlet
l3 = gmsh.model.geo.addLine(3, 4, 3)  # tank wall
l4 = gmsh.model.geo.addLine(4, 5, 4)  # tank wall
l5 = gmsh.model.geo.addLine(5, 6, 5)  # nozzle connection
l6 = gmsh.model.geo.addLine(6, 7, 6)  # nozzle inlet
l7 = gmsh.model.geo.addLine(7, 1, 7)  # nozzle axis

#..create surface
loop = gmsh.model.geo.addCurveLoop([1, 2, 3, 4, 5, 6, 7], 1)
surf = gmsh.model.geo.addPlaneSurface([1], 1)

#..apply transfinite constraints for high-quality 2D mesh
# Axial direction (streamwise) - this is critical for good revolution
gmsh.model.geo.mesh.setTransfiniteCurve(1, 20, "Progression", 1.05)  # tank axis
gmsh.model.geo.mesh.setTransfiniteCurve(7, 8)   # nozzle axis

# Radial direction (cross-stream) - ensures structured layers
gmsh.model.geo.mesh.setTransfiniteCurve(2, 6)   # tank outlet
gmsh.model.geo.mesh.setTransfiniteCurve(3, 12)  # tank wall
gmsh.model.geo.mesh.setTransfiniteCurve(4, 6)   # tank wall
gmsh.model.geo.mesh.setTransfiniteCurve(5, 12)  # nozzle connection
gmsh.model.geo.mesh.setTransfiniteCurve(6, 8)   # nozzle inlet

# Apply transfinite surface for structured mesh
gmsh.model.geo.mesh.setTransfiniteSurface(1)
gmsh.model.geo.mesh.setRecombine(2, 1)  # quadrilateral elements

#..3/9: synchronize before revolution
gmsh.model.geo.synchronize()

#..4/9: revolve to create 3D wedge
wedge_angle = 10.0 * π / 180.0  # 10 degrees for good 3D resolution
n_layers = 5  # number of layers in circumferential direction

println("Creating 3D wedge by revolution...")
println("Wedge angle: $(wedge_angle * 180 / π) degrees")
println("Circumferential layers: $(n_layers)")

# Create revolution with layers for better 3D structure
revolution_result = gmsh.model.geo.revolve(
    [(2, surf)],           # surface to revolve
    0, 0, 0,              # axis point (origin)
    1, 0, 0,              # axis directioi (x-axis)
    wedge_angle,          # angle
    [n_layers],           # number of layers
    [1.0]                 # relative height of layers
)

#..5/9: synchronize after revolution
gmsh.model.geo.synchronize()

#..6/9: get all entities and assign physical groups systematically
all_volumes = gmsh.model.getEntities(3)
all_surfaces = gmsh.model.getEntities(2)
all_curves = gmsh.model.getEntities(1)

println("Created entities:")
println("  Volumes: $(length(all_volumes))")
println("  Surfaces: $(length(all_surfaces))")
println("  Curves: $(length(all_curves))")

#..7/9: assign physical groups with better boundary identification
# Fluid volume
if length(all_volumes) > 0
    gmsh.model.addPhysicalGroup(3, [all_volumes[1][2]], -1, "fluid")
    println("Assigned fluid volume: tag $(all_volumes[1][2])")
end

# For 3D wedge, we need to identify surfaces more carefully
axis_surfaces = []
wall_surfaces = []
inlet_surfaces = []
outlet_surfaces = []
front_back_surfaces = []

for (dim, tag) in all_surfaces
    # Get surface center of mass and normal to identify boundary type
    try
        com = gmsh.model.occ.getCenterOfMass(dim, tag)
        
        if abs(com[2]) < 1e-6  # y ≈ 0 (axis surfaces)
            push!(axis_surfaces, tag)
        elseif com[1] < Lin + 1e-6  # inlet region (x near nozzle inlet)
            push!(inlet_surfaces, tag)
        elseif com[1] > L - 1e-6  # outlet region (x near tank outlet)
            push!(outlet_surfaces, tag)
        elseif abs(com[3]) < 1e-6 || abs(com[3] - wedge_angle*com[1]) < 1e-3  # front/back faces
            push!(front_back_surfaces, tag)
        else
            push!(wall_surfaces, tag)
        end
    catch
        # If getCenterOfMass fails, classify based on tag (backup method)
        if tag <= 2
            push!(axis_surfaces, tag)
        elseif tag >= length(all_surfaces) - 1
            push!(front_back_surfaces, tag)
        else
            push!(wall_surfaces, tag)
        end
    end
end

# Assign physical groups
if length(axis_surfaces) > 0
    gmsh.model.addPhysicalGroup(2, axis_surfaces, -1, "axis")
    println("Assigned axis surfaces: $(length(axis_surfaces))")
end
if length(wall_surfaces) > 0
    gmsh.model.addPhysicalGroup(2, wall_surfaces, -1, "wall")
    println("Assigned wall surfaces: $(length(wall_surfaces))")
end
if length(inlet_surfaces) > 0
    gmsh.model.addPhysicalGroup(2, inlet_surfaces, -1, "inlet")
    println("Assigned inlet surfaces: $(length(inlet_surfaces))")
end
if length(outlet_surfaces) > 0
    gmsh.model.addPhysicalGroup(2, outlet_surfaces, -1, "outlet")
    println("Assigned outlet surfaces: $(length(outlet_surfaces))")
end
if length(front_back_surfaces) >= 2
    gmsh.model.addPhysicalGroup(2, [front_back_surfaces[1]], -1, "front")
    gmsh.model.addPhysicalGroup(2, [front_back_surfaces[end]], -1, "back")
    println("Assigned front/back surfaces: $(length(front_back_surfaces))")
end

#..8/9: generate 3D mesh with optimization
gmsh.option.setNumber("Mesh.Algorithm3D", 4)      # Frontal algorithm
gmsh.option.setNumber("Mesh.Optimize", 1)         # Optimize mesh
gmsh.option.setNumber("Mesh.OptimizeNetgen", 1)   # Netgen optimization

println("Generating 3D mesh...")
gmsh.model.mesh.generate(3)

#..9/9: output and visualization
if (true) gmsh.write("tankAndNozzle_3D_wedge.msh") end 
if (true) gmsh.fltk.run() end 

# Print summary
println("\n=== 3D Wedge Mesh Summary ===")
println("Wedge angle: $(wedge_angle * 180 / π)°")
println("Circumferential layers: $(n_layers)")
println("Physical groups:")
println("  - Fluid volume: $(length(all_volumes)) volume(s)")
println("  - Axis surfaces: $(length(axis_surfaces))")
println("  - Wall surfaces: $(length(wall_surfaces))")
println("  - Inlet surfaces: $(length(inlet_surfaces))")
println("  - Outlet surfaces: $(length(outlet_surfaces))")
println("  - Front/back surfaces: $(length(front_back_surfaces))")
println("\nMesh file saved as: tankAndNozzle_3D_wedge.msh")

gmsh.finalize()

### Alternative: Simple 3D Wedge (Debugging Version)

Simpler approach since advanced one is not working right now:

In [9]:
# Simple 3D wedge for debugging
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("simple_3D_wedge")

# Create simple upper half geometry
L = 1.0; R = 0.1; lc = 0.02
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc, 1)    # axis origin
p2 = gmsh.model.geo.addPoint(L, 0, 0, lc, 2)    # axis end
p3 = gmsh.model.geo.addPoint(L, R, 0, lc, 3)    # corner
p4 = gmsh.model.geo.addPoint(0, R, 0, lc, 4)    # top

# Create lines and surface
l1 = gmsh.model.geo.addLine(1, 2, 1)  # axis
l2 = gmsh.model.geo.addLine(2, 3, 2)  # outlet
l3 = gmsh.model.geo.addLine(3, 4, 3)  # wall
l4 = gmsh.model.geo.addLine(4, 1, 4)  # wall

loop = gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
surf = gmsh.model.geo.addPlaneSurface([1], 1)

# Synchronize before revolution
gmsh.model.geo.synchronize()

# Simple revolution
angle = 10.0 * π / 180.0
println("Revolving around Z-axis by $(angle * 180 / π) degrees...")

# Revolve around Z-axis (0,0,1)
revolution_entities = gmsh.model.geo.revolve(
    [(2, surf)],    # surface to revolve
    0, 0, 0,        # point on axis
    1, 0, 0,        # axis direction (x-axis)
    angle           # angle
)

gmsh.model.geo.synchronize()

# Check what was created
volumes = gmsh.model.getEntities(3)
surfaces = gmsh.model.getEntities(2)
println("Revolution created:")
println("  Volumes: $(length(volumes))")
println("  Surfaces: $(length(surfaces))")

# Simple physical groups
if length(volumes) > 0
    gmsh.model.addPhysicalGroup(3, [volumes[1][2]], -1, "fluid")
end
if length(surfaces) > 0
    surface_tags = [s[2] for s in surfaces]
    gmsh.model.addPhysicalGroup(2, surface_tags, -1, "boundary")
end

# Generate mesh
println("Generating 3D mesh...")
try
    gmsh.model.mesh.generate(3)
    println("SUCCESS: 3D mesh generated!")
catch e
    println("3D mesh failed: $e")
    println("Generating 2D mesh on surfaces...")
    gmsh.model.mesh.generate(2)
end

# Save and visualize
gmsh.write("simple_3D_debug.msh")
gmsh.fltk.run()
gmsh.finalize()

Revolving around Z-axis by 10.0 degrees...
Revolution created:
  Volumes: 1
  Surfaces: 5
Generating 3D mesh...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 40%] Meshing curve 4 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 12 (Circle)
Info    : [ 90%] Meshing curve 16 (Circle)
Info    : Done meshing 1D (Wall 0.000995874s, CPU 0s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 13 (Surface, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 17 (Surface, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 20 (Surface, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 21 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0313892s, CPU 0s)
Info    : Meshing 3D...
Info    : 3D Meshing 1 volume

## Summary and Conclusions

This notebook has successfully demonstrated multiple approaches to mesh generation for tank-nozzle geometries with a focus on metal hydride storage applications.

(Finish later)